In [2]:
import pandas as pd
import numpy as np
import sqlalchemy as sa              #Para conexión a BD, requerido para usar pd.read_sql()
from sqlalchemy import text
import urllib                        #Para formatear string de conexión


param_DW = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=10.34.71.202;UID=datawarehouse;PWD=datawarehouse;DATABASE=Estudios;TrustServerCertificate=yes")
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % param_DW)

In [34]:
data_2024 = pd.read_csv("C:/o/DCCP/Javier Guajardo - Traspaso/2. Cálculo de Ahorro/savings_dccp/data/combustible/precios por estacion/energia_abierta/2024.csv/2024.csv", sep=",")

reemplazos = {
    'GLP':'GasLicuado'
}

def modify_data(data,fueltype, reemplazos):
    renamed_data = data.rename(columns={'codigo':'id','fecha_actualizacion':'fecha_inicio','combustible':'tipo'})
    renamed_data['fecha_fin'] = pd.Timestamp.now().strftime('%Y-%m-%d')
    reordered_data = renamed_data[['id','fecha_inicio','fecha_fin','precio','tipo','hora_actualizacion']]
    reordered_data['tipo'] = reordered_data['tipo'].replace(reemplazos)
    modified_data = reordered_data[reordered_data['tipo'].isin(reemplazos.values())]
    filtered_data = modified_data[modified_data['tipo']==fueltype]
    return filtered_data

Gas_24 = modify_data(data_2024, 'GasLicuado', reemplazos) 


C:\Users\hector.garrido\AppData\Local\Temp\ipykernel_20752\2281914040.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2024 = pd.read_csv("C:/o/DCCP/Javier Guajardo - Traspaso/2. Cálculo de Ahorro/savings_dccp/data/combustible/precios por estacion/energia_abierta/2024.csv/2024.csv", sep=",")
C:\Users\hector.garrido\AppData\Local\Temp\ipykernel_20752\2281914040.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reordered_data['tipo'] = reordered_data['tipo'].replace(reemplazos)


In [47]:
def create_table(new_table):
    with engine.connect() as conn: 
        trans = conn.begin()
        try: 
            # Crear la nueva tabla con la misma estructura que la tabla histórica
            conn.execute(
                text(f'''
                CREATE TABLE Estudios.dbo.{new_table}(
                    id VARCHAR(50),
                    fecha_inicio DATE,
                    fecha_fin DATE,
                    precio DECIMAL(10,2),
                    tipo VARCHAR(50)
                )
                     ''')
            )
            #Confirmar la transacción
            trans.commit()
            print(f"Tabla {new_table} creada exitosamente.")
        except Exception as e: 
            trans.rollback()
            print(f"Ha ocurrido un error al momento de crear la tabla {new_table}:{e}")
        finally: 
            conn.close()

def drop_table(old_table):
    with engine.connect() as conn:
        trans = conn.begin()
        try: 
            #Borra una tabla ya existente 
            conn.execute(
                text(
                    f'''
                    DROP TABLE Estudios.dbo.{old_table}
                    '''
                )
            )
            #Confirmar la transacción
            trans.commit()
            print(f"Tabla {old_table} borrada existosamente.")
        except Exception as e: 
            trans.rollback()
            print(f"Ha ocurrido un error al momento de borrar la tabla {old_table}:{e}")
        finally:
            conn.close()


In [46]:
#create_table('GasLicuado')
drop_table('GasLicuadoAnterior')
drop_table('GasLicuadoAux')
drop_table('GasLicuadoHist')


Tabla GasLicuadoAnterior borrada existosamente.
Tabla GasLicuadoAux borrada existosamente.
Tabla GasLicuadoHist borrada existosamente.


In [48]:
create_table('GasLicuadoAnterior')
create_table('GasLicuadoAux')
create_table('GasLicuadoHist')

Tabla GasLicuadoAnterior creada exitosamente.
Tabla GasLicuadoAux creada exitosamente.
Tabla GasLicuadoHist creada exitosamente.


In [49]:
with engine.connect() as conn:
    trans = conn.begin()
    conn.execute(
                text(
                    f'''
                    ALTER TABLE Estudios.dbo.GasLicuadoAux
                    ADD hora_actualizacion TIME;
                    '''
                )
            )
            #Confirmar la transacción
    trans.commit()

In [50]:
# Realiza respaldo de la tabla histórica en la tabla Anterior

#Modifica tabla auxilar en la que se irán cargando los nuevos datos

def cargar_y_modificar_aux(tabla_aux, datos):
    # Paso 1: Modificar la tabla auxiliar
    with engine.connect() as conn:
        trans = conn.begin()
        try:
            # Borrar los datos existentes
            conn.execute(text(f'''
            DELETE FROM Estudios.dbo.{tabla_aux}
            '''))

            # Alterar la tabla para añadir la columna 'hora_actualizacion'
            #conn.execute(text(f'''
            #ALTER TABLE {tabla_aux}
            #ADD hora_actualizacion TIME;
            #'''))
            
            trans.commit()
        except:
            trans.rollback()
            raise

    # Paso 2: Cargar los nuevos datos en la tabla auxiliar
    datos.to_sql(tabla_aux, con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date(),
        'hora_actualizacion': sa.types.Time()
    })


def update_table(table_ant,table_hist):
    with engine.connect() as conn:
        trans = conn.begin()
        try:
            # Eliminar los registros existentes en la tabla
            conn.execute(
                text(f'''
                DELETE FROM {table_ant}
                ''')
            )
            
            # Insertar los registros desde la tabla Histórica
            conn.execute(
                text(f'''
                INSERT INTO Estudios.dbo.{table_ant} (id, fecha_inicio, fecha_fin, precio, tipo)
                SELECT id, fecha_inicio, fecha_fin, precio, tipo
                FROM {table_hist}
                ''')
            )
            
            # Confirmar la transacción
            trans.commit()
        except Exception as e:
            trans.rollback()
            print(f"Ha ocurrido un error al momento de actualizar {table_ant}: {e}")
        finally:
            conn.close()




In [51]:
cargar_y_modificar_aux('GasLicuadoAux',Gas_24)

In [55]:
Gas_24[['id','precio','tipo','fecha_inicio','fecha_fin']].to_sql('GasLicuadoAnterior', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date()
    })

-1

In [56]:
Gas_24[['id','precio','tipo','fecha_inicio','fecha_fin']].to_sql('GasLicuadoHist', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date()
    })

-1